In [2]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split

from time import time
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
trainset = pd.read_csv("full_trainset_v2.csv")

In [4]:
#Date as index
df = trainset[trainset["Open"] == 1]
df = pd.DataFrame(df, columns=['Date','Store','Sales'])
df.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
df = df.reset_index(drop=True)
df = df.set_index('Date')

In [7]:
df.head()

,Store,Sales
Date,,
2013-01-01,1,0
2013-01-02,1,5530
2013-01-03,1,4327
2013-01-04,1,4486
2013-01-05,1,4997


In [32]:
import pmdarima as pm
dt = time()
mae_list = []
rmse_list = []
for i in range(len(df["Store"].unique())):
    try:
        # Filter by Store
        subset = df[(df["Store"]==(i+1))]
        subset = subset.drop(["Store"], axis = 1)

        # Split train and test set  
        train = subset.loc[:'2015-12-31', :]
        test = subset.loc['2015-01-01':, :]

        # Build Model
        model = pm.auto_arima(subset.Sales, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
        # Forecast
        n_periods = len(test)
        fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
        
        fc_series = pd.DataFrame(fc)
        fc_series.columns = ['Sales']
        fc_series = fc_series.reset_index(drop=True)
        fc_series = fc_series.set_index(test.index)
        test.columns = ['Sales']

        # Validate
        errors = abs(fc_series['Sales'] - test['Sales'])
        mae = round(np.mean(errors), 2)
        rmse = round(m.sqrt((1/len(test))*sum(errors**2)), 2)

        mae_list.append(mae)
        rmse_list.append(rmse)
    
    except:
        mae_list.append(np.nan)
        rmse_list.append(np.nan)
        pass

duration = time() - dt
print(duration)
print(rmse_list)

Fit ARIMA: order=(1, 0, 1); AIC=12542.222, BIC=12560.865, Fit time=0.142 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13028.092, BIC=13037.413, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=12540.651, BIC=12554.633, Fit time=0.043 seconds
Fit ARIMA: order=(0, 0, 1); AIC=12734.251, BIC=12748.233, Fit time=0.034 seconds
Fit ARIMA: order=(2, 0, 0); AIC=12542.063, BIC=12560.705, Fit time=0.056 seconds
Fit ARIMA: order=(2, 0, 1); AIC=12516.866, BIC=12540.169, Fit time=0.270 seconds
Fit ARIMA: order=(3, 0, 1); AIC=12505.614, BIC=12533.577, Fit time=0.304 seconds
Fit ARIMA: order=(3, 0, 0); AIC=12530.950, BIC=12554.253, Fit time=0.121 seconds
Fit ARIMA: order=(3, 0, 2); AIC=12493.037, BIC=12525.661, Fit time=0.387 seconds
Fit ARIMA: order=(2, 0, 2); AIC=12499.449, BIC=12527.413, Fit time=0.205 seconds
Fit ARIMA: order=(3, 0, 3); AIC=12499.144, BIC=12536.428, Fit time=0.541 seconds
Total fit time: 2.110 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13739.390, BIC=13758.048, Fit time=0.056 second

Fit ARIMA: order=(2, 0, 0); AIC=13008.162, BIC=13026.820, Fit time=0.092 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13011.840, BIC=13039.826, Fit time=0.227 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13010.046, BIC=13033.368, Fit time=0.113 seconds
Total fit time: 0.872 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13924.980, BIC=13943.638, Fit time=0.057 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14325.577, BIC=14334.905, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13970.037, BIC=13984.030, Fit time=0.057 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14138.509, BIC=14152.502, Fit time=0.028 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13857.870, BIC=13881.192, Fit time=0.344 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13905.562, BIC=13924.220, Fit time=0.158 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13853.618, BIC=13881.604, Fit time=0.356 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13855.714, BIC=13893.029, Fit time=0.332 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13910.618, BIC=13933.940, Fit time=0.077 second

Fit ARIMA: order=(2, 0, 1); AIC=13669.911, BIC=13693.168, Fit time=0.162 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13669.919, BIC=13693.176, Fit time=0.100 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13624.281, BIC=13652.190, Fit time=0.428 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13526.558, BIC=13559.119, Fit time=0.723 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13647.974, BIC=13675.884, Fit time=0.331 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13519.691, BIC=13556.904, Fit time=0.589 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13529.817, BIC=13562.378, Fit time=0.651 seconds
Total fit time: 3.099 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13553.693, BIC=13572.315, Fit time=0.178 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13848.114, BIC=13857.425, Fit time=0.005 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13553.584, BIC=13567.551, Fit time=0.054 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13615.114, BIC=13629.080, Fit time=0.065 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13553.153, BIC=13571.775, Fit time=0.167 second

Fit ARIMA: order=(1, 0, 1); AIC=13932.244, BIC=13950.876, Fit time=0.212 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14207.650, BIC=14216.966, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13933.212, BIC=13947.186, Fit time=0.015 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14017.930, BIC=14031.904, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13914.820, BIC=13938.110, Fit time=0.143 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13931.411, BIC=13950.043, Fit time=0.087 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13922.635, BIC=13950.583, Fit time=0.296 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13917.995, BIC=13950.601, Fit time=0.630 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13928.212, BIC=13956.160, Fit time=0.190 seconds
Total fit time: 1.591 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13268.564, BIC=13287.186, Fit time=0.062 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13616.036, BIC=13625.347, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13267.635, BIC=13281.601, Fit time=0.012 second

Fit ARIMA: order=(3, 0, 2); AIC=13481.627, BIC=13514.234, Fit time=0.634 seconds
Total fit time: 2.620 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13810.144, BIC=13828.776, Fit time=0.059 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13946.637, BIC=13955.953, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13825.858, BIC=13839.832, Fit time=0.078 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13864.102, BIC=13878.076, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13805.111, BIC=13828.401, Fit time=0.158 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13803.185, BIC=13821.817, Fit time=0.096 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13801.837, BIC=13829.785, Fit time=0.234 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13805.038, BIC=13828.328, Fit time=0.134 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13791.950, BIC=13824.556, Fit time=0.412 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13754.871, BIC=13782.819, Fit time=0.317 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13798.128, BIC=13830.734, Fit time=0.653 second

...

Fit ARIMA: order=(3, 0, 2); AIC=13991.334, BIC=14023.940, Fit time=0.428 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13989.446, BIC=14017.394, Fit time=0.171 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13968.494, BIC=14001.100, Fit time=0.485 seconds
Fit ARIMA: order=(1, 0, 2); AIC=14006.409, BIC=14029.699, Fit time=0.071 seconds
Fit ARIMA: order=(1, 0, 3); AIC=14005.401, BIC=14033.349, Fit time=0.196 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13962.744, BIC=14000.008, Fit time=0.857 seconds
Total fit time: 2.802 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13696.327, BIC=13714.959, Fit time=0.148 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13898.953, BIC=13908.269, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13741.074, BIC=13755.048, Fit time=0.059 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13800.113, BIC=13814.087, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13690.638, BIC=13713.928, Fit time=0.167 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13690.680, BIC=13709.312, Fit time=0.067 second

Fit ARIMA: order=(2, 0, 2); AIC=13567.479, BIC=13595.256, Fit time=0.358 seconds
Total fit time: 0.858 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13746.314, BIC=13764.951, Fit time=0.056 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14061.167, BIC=14070.486, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13764.497, BIC=13778.475, Fit time=0.011 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13846.990, BIC=13860.968, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13748.267, BIC=13771.564, Fit time=0.066 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13748.132, BIC=13771.428, Fit time=0.081 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13745.358, BIC=13773.314, Fit time=0.120 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13648.129, BIC=13680.744, Fit time=0.356 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13728.724, BIC=13756.679, Fit time=0.092 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13646.531, BIC=13683.806, Fit time=0.870 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13723.795, BIC=13756.410, Fit time=0.152 second

Fit ARIMA: order=(3, 0, 0); AIC=14129.366, BIC=14152.688, Fit time=0.097 seconds
Fit ARIMA: order=(3, 0, 2); AIC=14042.092, BIC=14074.743, Fit time=0.277 seconds
Fit ARIMA: order=(2, 0, 2); AIC=14040.481, BIC=14068.467, Fit time=0.410 seconds
Fit ARIMA: order=(2, 0, 3); AIC=14042.100, BIC=14074.750, Fit time=0.237 seconds
Fit ARIMA: order=(3, 0, 3); AIC=14012.555, BIC=14049.870, Fit time=0.553 seconds
Total fit time: 2.182 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13622.482, BIC=13641.104, Fit time=0.058 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13973.987, BIC=13983.298, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13638.809, BIC=13652.776, Fit time=0.042 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13688.740, BIC=13702.707, Fit time=0.022 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13617.905, BIC=13641.182, Fit time=0.135 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13617.491, BIC=13636.113, Fit time=0.064 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13619.929, BIC=13647.862, Fit time=0.199 second

Fit ARIMA: order=(3, 0, 3); AIC=14247.811, BIC=14285.126, Fit time=0.600 seconds
Total fit time: 2.483 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13109.220, BIC=13127.842, Fit time=0.055 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13368.363, BIC=13377.674, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13109.135, BIC=13123.101, Fit time=0.011 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13160.014, BIC=13173.981, Fit time=0.011 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13108.611, BIC=13127.233, Fit time=0.087 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13087.278, BIC=13110.556, Fit time=0.186 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13035.302, BIC=13067.890, Fit time=0.542 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13075.621, BIC=13103.554, Fit time=0.236 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13082.306, BIC=13110.239, Fit time=0.197 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13035.176, BIC=13072.420, Fit time=0.397 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13035.536, BIC=13068.124, Fit time=0.397 second

Fit ARIMA: order=(2, 0, 1); AIC=13745.214, BIC=13768.504, Fit time=0.202 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13743.481, BIC=13762.113, Fit time=0.021 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13746.430, BIC=13774.378, Fit time=0.163 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13744.783, BIC=13768.073, Fit time=0.030 seconds
Total fit time: 0.511 seconds
Fit ARIMA: order=(1, 0, 1); AIC=14643.367, BIC=14661.999, Fit time=0.055 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14852.520, BIC=14861.836, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=14648.099, BIC=14662.073, Fit time=0.012 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14710.419, BIC=14724.393, Fit time=0.011 seconds
Fit ARIMA: order=(2, 0, 1); AIC=14616.759, BIC=14640.049, Fit time=0.225 seconds
Fit ARIMA: order=(2, 0, 0); AIC=14640.383, BIC=14659.015, Fit time=0.022 seconds
Fit ARIMA: order=(2, 0, 2); AIC=14543.907, BIC=14571.855, Fit time=0.332 seconds
Fit ARIMA: order=(3, 0, 3); AIC=14529.401, BIC=14566.665, Fit time=1.055 second

In [33]:
rmse_list = pd.DataFrame(rmse_list, columns=["RMSE"])
mae_list = pd.DataFrame(mae_list, columns=["MAE"])

In [35]:
store = list(range(1,1116))
store_series = pd.DataFrame(store, columns=["Store"])
result = pd.merge(store_series, rmse_list ,how = 'left',left_index = True, right_index = True)
result = pd.merge(result, mae_list ,how = 'left',left_index = True, right_index = True)
result.head()

,Store,RMSE,MAE
0,1,846.32,701.79
1,2,1567.19,1282.10
2,3,2035.23,1706.18
3,4,1837.55,1391.57
4,5,1660.35,1320.16


In [36]:
result.to_csv("arima_result_final.csv", index=False)

In [37]:
result.RMSE.mean()

1788.575255605381

In [38]:
result.RMSE.median()

1694.88

In [19]:
import pmdarima as pm

"""To display up to 50 columns of dataset"""
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 8)

"""To display all outputs of each cell"""
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


np.random.seed(100)

"""To get a timing of each function decorated"""
def timing(f):
    """
    Decorator for timing functions
    Usage:
    @timing
    def function(&):
        pass
    """
    
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('function:%r took: %2.2f sec' % (f.__name__, end - start))
        return result
    return wrapper

class Trainer():
    def __init__(self, input_directory, df_): #input_directory = path to the folder containing df. 
        self.input_directory = input_directory
        self.df = df_
                
    @timing
    def read(self):
        """Read raw data"""
        self.df = pd.read_csv(self.input_directory + self.df)
    
    @timing
    def auto_arima(self, nb_store):
        #Date as index
        self.df = self.df[self.df["Open"] == 1]
        self.df = pd.DataFrame(self.df, columns=['Date','Store','Sales'])
        self.df.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
        #Re_indexation 
        self.df = self.df.reset_index(drop='True')
        
        """put the date as index"""
        self.df["Date"] = pd.to_datetime(self.df["Date"].astype(str), format = '%Y/%m/%d')
        self.df = self.df.set_index("Date")
        
        self.subset = self.df[(self.df["Store"]==(nb_store))]
        self.subset = self.subset.drop(["Store"], axis = 1)

        # Split train and test set  
        self.train = self.subset.loc[:'2015-12-31', :]
        self.test = self.subset.loc['2015-01-01':, :]

        # Build Model
        self.model = pm.auto_arima(self.subset.Sales, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
        # Forecast
        self.n_periods = len(self.test)
        self.fc, self.confint = self.model.predict(n_periods=self.n_periods, return_conf_int=True)

        self.fc_series = pd.DataFrame(self.fc)
        self.fc_series.columns = ['predicted_ARIMA']
        self.fc_series = self.fc_series.reset_index(drop=True)
        self.fc_series = self.fc_series.set_index(self.test.index)
        self.test.columns = ['Sales']

        # Validate
        self.errors = abs(self.fc_series['predicted_ARIMA'] - self.test['Sales'])
        self.mae = round(np.mean(self.errors), 2)
        self.rmse = round(m.sqrt((1/len(self.test))*sum(self.errors**2)), 2)

        self.result = pd.merge(self.test, self.fc_series ,how = 'left',left_index = True, right_index = True)
        self.result["error_ARIMA"] = abs(self.result['predicted_ARIMA'] - self.result['Sales'])

        return self.result

'To display up to 50 columns of dataset'

'To display all outputs of each cell'

'To get a timing of each function decorated'

In [23]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()

function:'read' took: 2.49 sec


In [24]:
result_351 = project.auto_arima(351)
result_351 = result_351.reset_index(drop=False)
result_351.head()

Fit ARIMA: order=(1, 0, 1); AIC=13434.473, BIC=13453.105, Fit time=0.143 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13667.225, BIC=13676.541, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13438.600, BIC=13452.574, Fit time=0.049 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13508.523, BIC=13522.497, Fit time=0.013 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13417.455, BIC=13440.745, Fit time=0.209 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13432.413, BIC=13451.045, Fit time=0.086 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13350.668, BIC=13378.616, Fit time=0.297 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13344.854, BIC=13382.118, Fit time=0.588 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13345.849, BIC=13378.455, Fit time=0.226 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13346.035, BIC=13378.641, Fit time=0.260 seconds
Total fit time: 1.878 seconds
function:'auto_arima' took: 2.42 sec


,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-01-02,6448,6461.756174,13.756174
1,2015-01-03,4848,6355.394180,1507.394180
2,2015-01-05,9569,5541.950601,4027.049399
3,2015-01-06,6950,5590.060379,1359.939621
4,2015-01-07,6447,5121.363557,1325.636443


In [25]:
result_351.to_csv("AutoARIMA_store351.csv", index=False)

In [26]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()
result_279 = project.auto_arima(279)
result_279 = result_279.reset_index(drop=False)
result_279.head()

function:'read' took: 2.50 sec
Fit ARIMA: order=(1, 0, 1); AIC=14344.704, BIC=14363.331, Fit time=0.151 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14635.887, BIC=14645.201, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=14343.355, BIC=14357.326, Fit time=0.010 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14411.547, BIC=14425.517, Fit time=0.011 seconds
Fit ARIMA: order=(2, 0, 0); AIC=14344.541, BIC=14363.167, Fit time=0.103 seconds
Fit ARIMA: order=(2, 0, 1); AIC=14341.024, BIC=14364.308, Fit time=0.191 seconds
Fit ARIMA: order=(3, 0, 1); AIC=14339.540, BIC=14367.480, Fit time=0.259 seconds
Fit ARIMA: order=(3, 0, 0); AIC=14342.608, BIC=14365.892, Fit time=0.179 seconds
Fit ARIMA: order=(3, 0, 2); AIC=14282.785, BIC=14315.382, Fit time=0.513 seconds
Fit ARIMA: order=(2, 0, 2); AIC=14337.835, BIC=14365.775, Fit time=0.324 seconds
Fit ARIMA: order=(3, 0, 3); AIC=14284.701, BIC=14321.955, Fit time=0.685 seconds
Total fit time: 2.436 seconds
function:'auto_arima' took: 2.94 sec


,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-01-02,7922,9823.546782,1901.546782
1,2015-01-03,9380,7954.952419,1425.047581
2,2015-01-05,18078,6931.545632,11146.454368
3,2015-01-07,12686,6715.648262,5970.351738
4,2015-01-08,11349,7218.751534,4130.248466


In [27]:
result_279.to_csv("AutoARIMA_store279.csv", index=False)

In [28]:
project = Trainer("/Users/rfdanti/Documents/UoM/2_data_analytics/coursework/CODE/", "full_trainset_v2.csv")
project.read()
result_733 = project.auto_arima(733)
result_733 = result_733.reset_index(drop=False)
result_733.head()

function:'read' took: 2.50 sec
Fit ARIMA: order=(1, 0, 1); AIC=16474.152, BIC=16493.544, Fit time=0.120 seconds
Fit ARIMA: order=(0, 0, 0); AIC=16837.285, BIC=16846.981, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=16501.866, BIC=16516.410, Fit time=0.040 seconds
Fit ARIMA: order=(0, 0, 1); AIC=16608.054, BIC=16622.598, Fit time=0.020 seconds
Fit ARIMA: order=(2, 0, 1); AIC=16476.124, BIC=16500.364, Fit time=0.136 seconds
Fit ARIMA: order=(1, 0, 2); AIC=16476.093, BIC=16500.333, Fit time=0.227 seconds
Fit ARIMA: order=(2, 0, 2); AIC=16469.791, BIC=16498.879, Fit time=0.324 seconds
Fit ARIMA: order=(3, 0, 2); AIC=16348.230, BIC=16382.166, Fit time=0.775 seconds
Fit ARIMA: order=(3, 0, 1); AIC=16471.369, BIC=16500.457, Fit time=0.254 seconds
Fit ARIMA: order=(3, 0, 3); AIC=16314.638, BIC=16353.422, Fit time=5.940 seconds
Fit ARIMA: order=(2, 0, 3); AIC=16452.207, BIC=16486.143, Fit time=0.311 seconds
Total fit time: 8.155 seconds
function:'auto_arima' took: 8.67 sec


,Date,Sales,predicted_ARIMA,error_ARIMA
0,2015-01-01,12566,14830.482616,2264.482616
1,2015-01-02,14651,15379.614954,728.614954
2,2015-01-03,13044,15962.912555,2918.912555
3,2015-01-04,16040,14899.803468,1140.196532
4,2015-01-05,14797,14594.165672,202.834328


In [29]:
result_733.to_csv("AutoARIMA_store733.csv", index=False)

In [5]:
import pmdarima as pm
dt = time()
mae_list = []
rmse_list = []
for i in range(len(df["Store"].unique())):
    try:
        # Filter by Store
        subset = df[(df["Store"]==(i+1))]
        subset = subset.drop(["Store"], axis = 1)

        # Split train and test set  
        train = subset.loc[:'2015-06-13', :]
        test = subset.loc['2015-06-14':, :]

        # Build Model
        model = pm.auto_arima(subset.Sales, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
        # Forecast
        n_periods = len(test)
        fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
        
        fc_series = pd.DataFrame(fc)
        fc_series.columns = ['Sales']
        fc_series = fc_series.reset_index(drop=True)
        fc_series = fc_series.set_index(test.index)
        test.columns = ['Sales']

        # Validate
        errors = abs(fc_series['Sales'] - test['Sales'])
        mae = round(np.mean(errors), 2)
        rmse = round(m.sqrt((1/len(test))*sum(errors**2)), 2)

        mae_list.append(mae)
        rmse_list.append(rmse)
    
    except:
        mae_list.append(np.nan)
        rmse_list.append(np.nan)
        pass

duration = time() - dt
print(duration)
print(rmse_list)

Fit ARIMA: order=(1, 0, 1); AIC=12542.222, BIC=12560.865, Fit time=0.151 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13028.092, BIC=13037.413, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=12540.651, BIC=12554.633, Fit time=0.053 seconds
Fit ARIMA: order=(0, 0, 1); AIC=12734.251, BIC=12748.233, Fit time=0.081 seconds
Fit ARIMA: order=(2, 0, 0); AIC=12542.063, BIC=12560.705, Fit time=0.151 seconds
Fit ARIMA: order=(2, 0, 1); AIC=12516.866, BIC=12540.169, Fit time=0.474 seconds
Fit ARIMA: order=(3, 0, 1); AIC=12505.614, BIC=12533.577, Fit time=0.287 seconds
Fit ARIMA: order=(3, 0, 0); AIC=12530.950, BIC=12554.253, Fit time=0.225 seconds
Fit ARIMA: order=(3, 0, 2); AIC=12493.037, BIC=12525.661, Fit time=0.471 seconds
Fit ARIMA: order=(2, 0, 2); AIC=12499.449, BIC=12527.413, Fit time=0.304 seconds
Fit ARIMA: order=(3, 0, 3); AIC=12499.144, BIC=12536.428, Fit time=0.638 seconds
Total fit time: 2.859 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13739.390, BIC=13758.048, Fit time=0.086 second

Fit ARIMA: order=(0, 0, 1); AIC=13061.134, BIC=13075.127, Fit time=0.019 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13010.105, BIC=13033.427, Fit time=0.270 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13008.162, BIC=13026.820, Fit time=0.087 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13011.840, BIC=13039.826, Fit time=0.326 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13010.046, BIC=13033.368, Fit time=0.149 seconds
Total fit time: 1.046 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13924.980, BIC=13943.638, Fit time=0.075 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14325.577, BIC=14334.905, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13970.038, BIC=13984.031, Fit time=0.066 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14138.508, BIC=14152.501, Fit time=0.093 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13857.870, BIC=13881.192, Fit time=0.512 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13905.562, BIC=13924.220, Fit time=0.118 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13853.618, BIC=13881.604, Fit time=0.254 second

Fit ARIMA: order=(2, 0, 1); AIC=13669.911, BIC=13693.169, Fit time=0.196 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13669.919, BIC=13693.176, Fit time=0.153 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13624.281, BIC=13652.190, Fit time=0.599 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13526.558, BIC=13559.119, Fit time=0.629 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13647.974, BIC=13675.884, Fit time=0.486 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13519.691, BIC=13556.904, Fit time=0.676 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13529.817, BIC=13562.378, Fit time=0.515 seconds
Total fit time: 3.436 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13553.693, BIC=13572.315, Fit time=0.181 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13848.114, BIC=13857.425, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13553.584, BIC=13567.551, Fit time=0.079 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13615.114, BIC=13629.080, Fit time=0.097 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13553.153, BIC=13571.775, Fit time=0.127 second

Fit ARIMA: order=(1, 0, 1); AIC=13932.244, BIC=13950.876, Fit time=0.085 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14207.650, BIC=14216.966, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13933.212, BIC=13947.186, Fit time=0.015 seconds
Fit ARIMA: order=(0, 0, 1); AIC=14017.930, BIC=14031.904, Fit time=0.018 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13914.819, BIC=13938.109, Fit time=0.255 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13931.411, BIC=13950.043, Fit time=0.159 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13922.635, BIC=13950.583, Fit time=0.429 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13917.995, BIC=13950.601, Fit time=0.834 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13928.212, BIC=13956.160, Fit time=0.329 seconds
Total fit time: 2.134 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13268.564, BIC=13287.186, Fit time=0.094 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13616.036, BIC=13625.347, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13267.635, BIC=13281.601, Fit time=0.015 second

Fit ARIMA: order=(3, 0, 2); AIC=13481.627, BIC=13514.234, Fit time=0.763 seconds
Total fit time: 4.191 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13810.142, BIC=13828.774, Fit time=0.168 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13946.637, BIC=13955.953, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13825.858, BIC=13839.832, Fit time=0.039 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13864.102, BIC=13878.076, Fit time=0.021 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13805.111, BIC=13828.401, Fit time=0.233 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13803.185, BIC=13821.817, Fit time=0.129 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13801.837, BIC=13829.785, Fit time=0.353 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13805.038, BIC=13828.328, Fit time=0.194 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13791.950, BIC=13824.556, Fit time=0.908 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13754.871, BIC=13782.819, Fit time=0.547 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13798.128, BIC=13830.734, Fit time=0.444 second

...

Fit ARIMA: order=(2, 0, 2); AIC=13131.332, BIC=13159.265, Fit time=0.331 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13119.911, BIC=13157.155, Fit time=0.742 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13131.356, BIC=13163.944, Fit time=0.407 seconds
Total fit time: 2.742 seconds
Fit ARIMA: order=(1, 0, 1); AIC=12824.150, BIC=12842.772, Fit time=0.166 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13130.207, BIC=13139.518, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=12822.730, BIC=12836.697, Fit time=0.088 seconds
Fit ARIMA: order=(0, 0, 1); AIC=12922.342, BIC=12936.308, Fit time=0.017 seconds
Fit ARIMA: order=(2, 0, 0); AIC=12823.902, BIC=12842.524, Fit time=0.107 seconds
Fit ARIMA: order=(2, 0, 1); AIC=12826.028, BIC=12849.305, Fit time=0.260 seconds
Total fit time: 0.646 seconds
Fit ARIMA: order=(1, 0, 1); AIC=14219.185, BIC=14237.832, Fit time=0.100 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14280.947, BIC=14290.271, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=14221.348, BIC=14

Fit ARIMA: order=(2, 0, 2); AIC=12838.103, BIC=12866.051, Fit time=0.436 seconds
Fit ARIMA: order=(3, 0, 3); AIC=12790.061, BIC=12827.326, Fit time=1.367 seconds
Fit ARIMA: order=(2, 0, 3); AIC=12849.721, BIC=12882.327, Fit time=0.342 seconds
Total fit time: 4.206 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13338.108, BIC=13356.740, Fit time=0.081 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13373.243, BIC=13382.559, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13353.511, BIC=13367.485, Fit time=0.013 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13359.792, BIC=13373.766, Fit time=0.014 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13322.390, BIC=13345.680, Fit time=0.089 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13322.078, BIC=13340.710, Fit time=0.146 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13319.670, BIC=13347.618, Fit time=0.302 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13319.371, BIC=13342.661, Fit time=0.173 seconds
Total fit time: 0.824 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13876.201, BIC=13

Fit ARIMA: order=(2, 0, 0); AIC=13271.123, BIC=13289.780, Fit time=0.121 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13244.039, BIC=13267.361, Fit time=0.398 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13238.305, BIC=13270.956, Fit time=0.785 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13231.575, BIC=13259.561, Fit time=0.530 seconds
Fit ARIMA: order=(2, 0, 3); AIC=13211.252, BIC=13243.903, Fit time=0.717 seconds
Fit ARIMA: order=(1, 0, 2); AIC=13269.090, BIC=13292.412, Fit time=0.329 seconds
Fit ARIMA: order=(1, 0, 3); AIC=13269.789, BIC=13297.776, Fit time=0.381 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13222.623, BIC=13259.939, Fit time=0.922 seconds
Total fit time: 4.496 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13656.147, BIC=13674.779, Fit time=0.091 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13707.999, BIC=13717.315, Fit time=0.003 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13663.769, BIC=13677.743, Fit time=0.015 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13672.959, BIC=13686.933, Fit time=0.018 second

Fit ARIMA: order=(2, 0, 1); AIC=16753.330, BIC=16777.570, Fit time=0.479 seconds
Fit ARIMA: order=(1, 0, 2); AIC=16753.347, BIC=16777.587, Fit time=0.504 seconds
Fit ARIMA: order=(2, 0, 2); AIC=16713.626, BIC=16742.714, Fit time=0.942 seconds
Fit ARIMA: order=(3, 0, 2); AIC=16700.296, BIC=16734.232, Fit time=0.855 seconds
Fit ARIMA: order=(3, 0, 1); AIC=16723.937, BIC=16753.025, Fit time=0.764 seconds
Fit ARIMA: order=(3, 0, 3); AIC=nan, BIC=nan, Fit time=nan seconds
Total fit time: 4.344 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13790.965, BIC=13809.602, Fit time=0.094 seconds
Fit ARIMA: order=(0, 0, 0); AIC=13793.132, BIC=13802.451, Fit time=0.002 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13794.011, BIC=13807.989, Fit time=0.021 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13794.257, BIC=13808.235, Fit time=0.015 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13781.810, BIC=13805.107, Fit time=0.110 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13780.300, BIC=13798.937, Fit time=0.029 seconds
Fit ARIMA: o

Fit ARIMA: order=(3, 0, 2); AIC=13403.213, BIC=13435.864, Fit time=0.593 seconds
Total fit time: 2.736 seconds
Fit ARIMA: order=(1, 0, 1); AIC=13835.350, BIC=13853.977, Fit time=0.112 seconds
Fit ARIMA: order=(0, 0, 0); AIC=14021.377, BIC=14030.691, Fit time=0.008 seconds
Fit ARIMA: order=(1, 0, 0); AIC=13833.511, BIC=13847.481, Fit time=0.053 seconds
Fit ARIMA: order=(0, 0, 1); AIC=13877.148, BIC=13891.118, Fit time=0.019 seconds
Fit ARIMA: order=(2, 0, 0); AIC=13835.260, BIC=13853.886, Fit time=0.122 seconds
Fit ARIMA: order=(2, 0, 1); AIC=13795.794, BIC=13819.078, Fit time=0.343 seconds
Fit ARIMA: order=(3, 0, 1); AIC=13761.616, BIC=13789.556, Fit time=0.327 seconds
Fit ARIMA: order=(3, 0, 0); AIC=13824.789, BIC=13848.073, Fit time=0.134 seconds
Fit ARIMA: order=(3, 0, 2); AIC=13692.793, BIC=13725.390, Fit time=0.629 seconds
Fit ARIMA: order=(2, 0, 2); AIC=13695.906, BIC=13723.847, Fit time=0.365 seconds
Fit ARIMA: order=(3, 0, 3); AIC=13682.120, BIC=13719.374, Fit time=0.820 second

In [6]:
rmse_list = pd.DataFrame(rmse_list, columns=["RMSE"])
mae_list = pd.DataFrame(mae_list, columns=["MAE"])

In [7]:
store = list(range(1,1116))
store_series = pd.DataFrame(store, columns=["Store"])
result = pd.merge(store_series, rmse_list ,how = 'left',left_index = True, right_index = True)
result = pd.merge(result, mae_list ,how = 'left',left_index = True, right_index = True)
result.head()

,Store,RMSE,MAE
0,1,790.88,662.84
1,2,1658.47,1292.15
2,3,2222.30,1936.03
3,4,1660.91,1318.34
4,5,1704.03,1345.11


In [8]:
result.RMSE.mean()

1902.8071210762323

In [9]:
result.RMSE.median()

1769.91

In [ ]:
result.to_csv("autoarima_result_6weeks.csv", index=False)